<a href="https://colab.research.google.com/github/naji-s/DeepRL/blob/master/HW1_Blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10-703: Homework 1 - Behavior Cloning, DAGGER, CMA-ES, and GAIL

You will implement this assignment right here in this Colab notebook. Colab is a Jupyter notebook that runs in the cloud. If you haven't used Colab before, we recommend checking out the following tutorial:
https://colab.sandbox.google.com/notebooks/welcome.ipynb.
Note that all cells modify the same global state, so imported packages as well as functions and variables declared in one cell will be accessible in other cells.


To get started, click the ``Open in Playground`` button in the upper right. Then click the ``Copy to Drive`` button in the upper center to save a copy in your Google drive. In the future, you will be able to find the notebook by looking in your Google drive folder.

Now, you're ready start coding. You will want to run each cell in this notebook by clicking the "play" button to the left of the cell (or using [ctrl -> enter]. Look for ``WRITE CODE HERE'' to identify places where you need to write some code. Each section involves writing 3 - 10 lines of code. 

When you're done, copy plots genetated by your code into the solution boxes in the submission LaTeX file released with this assignment. In addition to uploading your PDF submission to GradeScope (one per group), you should also upload your code. To do this, explort the code from this notebook (File -> Export.py), and then upload the .py file to GradeScope.


# Preliminaries
In these first few cells, you will implement some compoments that will be used for all problems.


### Setup: Import Dependencies

In [0]:
from collections import OrderedDict 
import gym
import keras
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import random
import tensorflow as tf

Using TensorFlow backend.


### Make the TF Model
We'll use the same architecture for each of the problems. By implementing a function that creates the model here, you won't need to implement it again for each problem.

In [0]:
def make_model():
  model = Sequential()      
  # WRITE CODE HERE
  # Add layers to the model:
  # a fully connected layer with 10 units
  # a tanh activation
  # another fully connected layer with 2 units (the number of actions)
  # a softmax activation (so the output is a proper distribution)
  model.compile(loss='categorical_crossentropy',
                     optimizer=tf.train.AdamOptimizer(),
                     metrics=['accuracy'])
  
  # We expect the model to have four weight variables (a kernel and bias for
  # both layers)
  assert len(model.weights) == 4, 'Model should have 4 weights.'
  return model

### Test the model
To confirm that the model is correct, we'll use it to solve a binary classification problem. The target function $f: \mathbb{R}^4 \rightarrow {0, 1}$ indicates whether the sum of the vector coordinates is positive:
$$f(x) = \delta \left(\sum_{i=1}^4 x_i > 0 \right)$$

You should achieve an accuracy of at least 98%.

In [0]:
model = make_model()
for t in range(20):
  X = np.random.normal(size=(1000, 4))  # some random data
  is_positive = np.sum(X, axis=1) > 0  # A simple binary function
  Y = np.zeros((1000, 2))
  Y[np.arange(1000), is_positive.astype(int)] = 1  # one-hot labels
  history = model.fit(X, Y, epochs=10, batch_size=256, verbose=0)
  loss = history.history['loss'][-1]
  acc = history.history['acc'][-1]
  print('(%d) loss= %.3f; accuracy = %.1f%%' % (t, loss, 100 * acc))

### Interacting with the Gym
Implement the function below for gathering an episode (a "rollout"). The environment we will use will implement the OpenAI Gym interface. For documentation, please see the link below:
http://gym.openai.com/docs/#environments

In [0]:
def action_to_one_hot(env, action):
    action_vec = np.zeros(env.action_space.n)
    action_vec[action] = 1
    return action_vec    
      
      
def generate_episode(env, policy):
  """Collects one rollout from the policy in an environment. The environment
  should implement the OpenAI Gym interface. A rollout ends when done=True. The
  number of states and actions should be the same, so you should not include
  the final state when done=True.

  Args:
    env: an OpenAI Gym environment.
    policy: a keras model
  Returns:
    states: a list of states visited by the agent.
    actions: a list of actions taken by the agent. While the original actions
      are discrete, it will be helpful to use a one-hot encoding. The actions
      that you return should be one-hot vectors (use action_to_one_hot())
    rewards: the reward received by the agent at each step.
  """
  done = False
  state = env.reset()

  states = []
  actions = []
  rewards = []
  while not done:
      # WRITE CODE HERE
  return np.array(states), np.array(actions), np.array(rewards)

### Test the data collection
Run the following cell and make sure you see "Test passed!"

In [0]:
# Create the environment.
env = gym.make('CartPole-v0')
policy = make_model()
states, actions, rewards = generate_episode(env, policy)
assert len(states) == len(actions), 'Number of states and actions should be equal.'
assert len(actions) == len(rewards), 'Number of actions and rewards should be equal.'
assert len(actions[0]) == 2, 'Actions should use one-hot encoding.'
print('Test passed!')

### Download the expert policy
Run the cell below to upload the expert policy (`expert.h5`) to the Colab runtime.

In [0]:
!wget https://raw.githubusercontent.com/cmudeeprl/703website/master/assets/homework/hw1/expert.h5

Status: OK


## Problem 1: Behavior Cloning and DAGGER (50 pt)

### Implementing Behavior Cloning and DAGGER
To implement behavior cloning and DAGGER, fill in the missing blocks of code below. The provided code loads an expert model upon creation of the `Imitation` class. The function `generate_behavior_cloning_data()` fills in `self._train_states` and `self._train_actions` with states and actions from a single episode. Later, when implementing DAGGER, you will finish implementing `generate_dagger_data()`.

In [0]:
class Imitation():

    def __init__(self, env, num_episodes):
        self.env = env
        self.expert = tf.keras.models.load_model('expert.h5')
        self.num_episodes = num_episodes
        
        
        self.model = make_model()
        
    def generate_behavior_cloning_data(self):
        self._train_states = []
        self._train_actions = []
        for _ in range(self.num_episodes):
            states, actions, rewards = generate_episode(self.env, self.expert)
            self._train_states.extend(states)
            self._train_actions.extend(actions)
        self._train_states = np.array(self._train_states)
        self._train_actions = np.array(self._train_actions)
        
    def generate_dagger_data(self):
        # WRITE CODE HERE
        # You should collect states and actions from the student policy
        # (self.model), and then relabel the actions using the expert policy.
        # This method does not return anything.
        
    def train(self, num_epochs=200):
        """Trains the model on training data generated by the expert policy.
        Args:
          env: The environment to run the expert policy on.
          num_epochs: number of epochs to train on the data generated by the expert.
        Return:
          loss: (float) final loss of the trained policy.
          acc: (float) final accuracy of the trained policy
        """
        # WRITE CODE HERE
        return loss, acc


    def evaluate(self, policy, n_episodes=50):
        rewards = []
        for i in range(n_episodes):
            _, _, r = generate_episode(self.env, policy)
            rewards.append(sum(r))
        r_mean = np.mean(rewards)
        return r_mean

### Experiment: Student vs Expert
In the next two cells, you will compare the performance of the expert policy
to the imitation policies obtained via behavior cloning and DAGGER.

In [0]:
# Uncomment one of the two lines below to select whether to run behavior
# cloning or dagger
mode = 'behavior cloning'
# mode = 'dagger'

num_episodes = 100  # Leave this fixed for now. You will experiment with
                    # changing it later.
num_iterations = 10  # Number of training iterations. Use a small number
                     # (e.g., 10) for debugging, and then try a larger number
                     # (e.g., 100).

# Create the environment.
env = gym.make('CartPole-v0')
im = Imitation(env, num_episodes)
expert_reward = im.evaluate(im.expert)
print('Expert reward: %.2f' % expert_reward)

loss_vec = []
acc_vec = []
imitation_reward_vec = []
for t in range(num_iterations):
  if mode == 'behavior cloning':
    im.generate_behavior_cloning_data()
  elif mode == 'dagger':
    im.generate_dagger_data()
  else:
    raise ValueError('Unknown mode: %s' % mode)
  loss, acc = im.train(num_epochs=1)
  imitation_reward = im.evaluate(im.model)
  loss_vec.append(loss)
  acc_vec.append(acc)
  imitation_reward_vec.append(imitation_reward)
  print('(%d) loss = %.3f; accuracy = %.2f; reward = %.1f' % (t, loss, acc, imitation_reward))

### Plot the results
After saving your plots by running `plt.savefig(FILENAME)`, you can download them by navigating to the `Files` tab on the left, and then right-clicking on each filename and selecting `Download`.

In [0]:
### Plot the results
plt.figure(figsize=(12, 3))
plt.subplot(131)
plt.title('Reward')
plt.plot(imitation_reward_vec, label='imitation')
plt.hlines(expert_reward, 0, len(imitation_reward_vec), label='expert')
plt.xlabel('iterations')
plt.ylabel('return')
plt.legend()
plt.ylim([0, None])

plt.subplot(132)
plt.title('Loss')
plt.plot(loss_vec)
plt.xlabel('iterations')
plt.ylabel('loss')

plt.subplot(133)
plt.title('Accuracy')
plt.plot(acc_vec)
plt.xlabel('iterations')
plt.ylabel('accuracy')
plt.tight_layout()
plt.savefig('student_vs_expert_%s.png' % mode, dpi=300)
plt.show()

### Experiment: How much expert data is needed?
This question studies how the amount of expert data effects the performance. You will run the same experiment as above, each time varying the number of expert episodes collected at each iteration. Use values of 1, 10, 50, and 100. You can keep the number of iterations fixed at 100.

In [0]:
random_seeds = 5
# Dictionary mapping number of expert trajectories to a list of rewards.
# Each is the result of running with a different random seed.
reward_data = OrderedDict({  
    1: [],
    10: [],
    50: [],
    100: []
})
accuracy_data = OrderedDict({  
    1: [],
    10: [],
    50: [],
    100: []
})
loss_data = OrderedDict({  
    1: [],
    10: [],
    50: [],
    100: []
})
for num_episodes in [1, 10, 50, 100]:
  for t in range(random_seeds):
    print('num_episodes: %s; seed: %d' % (num_episodes, t))
    # WRITE CODE HERE
    # Hint: The code here should be nearly identical to code after the
    # "Student vs Expert" cell. Feel free to copy and paste.

Plot the reward, loss, and accuracy for each, remembering to label each line.

In [0]:
keys = [1, 10, 50, 100]
plt.figure(figsize=(12, 4))
for (index, (data, name)) in enumerate(zip([reward_data, accuracy_data, loss_data],
                                           ['reward', 'accuracy', 'loss'])):
  plt.subplot(1, 3, index + 1)
  plt.plot(data.keys(), data.values(), '-o')
  plt.xlabel('number of expert trajectories', fontsize=16)
  plt.ylabel('name', fontsize=16)
plt.savefig('expert_data_%s.png' % mode, dpi=300)
plt.show()

## Problem 2: CMA-ES (25 pt)
In this section, you will implement CMA-ES, a black-box optimization algorithm. You will then use CMA-ES to solve a reinforcement learning problem. In particular, the function you will maximize is
$$\max_\theta J(\theta) = E_{\pi_\theta} \left[ \sum_t r(s_t, a_t) \right]$$

In [0]:
class CMAES:
    def __init__(self, env, L, n, p, sigma, noise, reward_fn=None):
        """
        Args:
          env: environment with which to evaluate different weights
          L: number of episodes used to evaluate a given set of weights
          n: number of members (weights) in each generation
          p: proportion of members used to update the mean and covariance
          sigma: initial std
          noise: additional noise to add to covariance
          reward_fn: if specified, this reward function is used instead of the 
            default environment reward. This will be used in Problem 3, when the
            reward function will come from the discriminator. The reward
            function should be a function of the state and action.
        """

        self.env = env
        self.model = make_model()
        # Let d be the dimension of the 1d vector of weights.
        self.d = sum(int(np.product(w.shape)) for w in self.model.weights)
        self.mu = np.zeros(self.d)
        self.S = sigma**2 * np.eye(self.d)

        self.L = L
        self.n = n
        self.p = p
        self.noise = noise
        self.reward_fn = reward_fn


    def populate(self):
        """
        Populate a generation using the current estimates of mu and S
        """
        self.population = []
        # WRITE CODE HERE

    def set_weight(self, member):
        ind = 0
        weights = []
        for w in self.model.weights:
          if len(w.shape) > 1:
            mat = member[ind:ind+int(w.shape[0]*w.shape[1])]
            mat = np.reshape(mat, w.shape)
            ind += int(w.shape[0]*w.shape[1])
          else:
            mat = member[ind:ind+int(w.shape[0])]
            ind += int(w.shape[0])
          weights.append(mat)

        self.model.set_weights(weights)
        

    def evaluate(self, member, num_episodes):
        """
        Evaluate a set of weights by interacting with the environment and
        return the average total reward over num_episodes.
        """
        self.set_weight(member)
        return self.evaluate_policy(self.model, num_episodes)
    
    def evaluate_policy(self, policy, num_episodes):
        episode_rewards = []
        for episode in range(num_episodes):
          # WRITE CODE HERE
        return np.mean(episode_rewards)

    def train(self):
        """
        Perform an iteration of CMA-ES by evaluating all members of the
        current population and then updating mu and S with the top self.p
        proportion of members. Note that self.populate automatically deletes
        all the old members, so you don't need to worry about deleting the
        "bad" members.

        """
        self.populate()
        # WRITE CODE HERE


        best_r = self.evaluate(best_member, 10)
        mu_r = self.evaluate(self.mu, 10)
        return mu_r, best_r



In [0]:
iterations = 10  # Use 10 for debugging, and then try 200 once you've got it working.
pop_size_vec = [50]             # Start with a population size of 50. Once that
# pop_size_vec = [20, 50, 100]  # works, try varying the population size.
                              
data = {pop_size: [] for pop_size in pop_size_vec}

for pop_size in pop_size_vec:
  print('Population size:', pop_size)
  env = gym.make('CartPole-v0')
  optimizer = CMAES(env,
                    L=1,  # number of episodes for evaluation
                    n=pop_size,  # population size
                    p=0.25,  # proportion of population to keep
                    sigma=10,  # initial std dev
                    noise=0.25)  # noise

  for t in range(iterations):
      mu_r, best_r = optimizer.train()
      data[pop_size].append((mu_r, best_r))
      print('(%d) avg member rew = %.2f; best member rew = %.2f' % (t, mu_r, best_r))

In [0]:
plt.figure(figsize=(12, 4))
plt.subplot(121)  # Left plot will show performance vs number of iterations
for pop_size, values in data.items():
  mu_r = np.array(values)[:, 1]  # Use the performance of the best point
  x = np.arange(len(mu_r)) + 1
  plt.plot(x, mu_r, label=str(pop_size))
  plt.ylabel('average return', fontsize=16)
  plt.xlabel('num. iterations', fontsize=16)

plt.subplot(122)  # Right plot will show performance vs number of points evaluated
for pop_size, values in data.items():
  mu_r = np.array(values)[:, 1]  # Use the performance of the best point
  x = pop_size * (np.arange(len(mu_r)) + 1)
  plt.plot(x, mu_r, label=str(pop_size))
  plt.ylabel('average return', fontsize=16)
  plt.xlabel('num. points evaluated', fontsize=16)

plt.legend()
plt.tight_layout()
plt.savefig('cmaes_pop_size.png')
plt.show()

## Problem 3: GAIL (25 pt)
For this problem, we will only condition the discriminator on the state, not the action.

In [0]:
class GAIL(object):
  
  def __init__(self, env):
    self.env = env
    self.expert = tf.keras.models.load_model('expert.h5')
    self.model = make_model()
    self.discriminator = make_model()
    self.cmaes = CMAES(env,
                  L=1,  # number of episodes for evaluation
                  n=20,  # population size
                  p=0.25,  # proportion of population to keep
                  sigma=10,  # initial std dev
                  noise=0.25,  # noise
                  reward_fn=self._reward_fn)
    self.expert_states = []
    self.student_states = []
    
  def _reward_fn(self, s, a):
    """Log probability that state is from expert."""
    del a
    p_expert = self.discriminator.predict(s[None])[0][0]
    return np.log(p_expert)
    
  def collect_data(self, num_episodes):
    """Collect data from the expert and imitation policy. After the initial
    iteration, there is no need to collect new data from the expert, as the
    expert policy never changes.
    """
    collect_expert = len(self.expert_states) == 0
    self.student_states = []
    for _ in range(num_episodes):
      # WRITE CODE HERE
      # Collect data from the expert policy
      # Collect data from the student policy
  
  def train_discriminator(self):
    # WRITE CODE HERE
    
    assert Y.shape[1] == 2  # Use a 1-hot encoding for the labels
    assert np.all(np.sum(Y, axis=1) == 1)
    history = self.discriminator.fit(X, Y, epochs=10, batch_size=256, verbose=0)
    loss = history.history['loss'][-1]
    acc = history.history['acc'][-1]
    return loss, acc
  
  def train_policy(self):
    mu_r, best_r = self.cmaes.train()
    return mu_r, best_r

In [0]:
### Implement the total variation distance to compare two policies
def get_x_position_histogram(states):
  x_vec = [s[0] for s in states]  # The x position is the first coordinate
  bins = np.linspace(-2.4, 2.4, 11)  # Need 11 edges to make 10 bins
  hist, _ = np.histogram(x_vec, bins=bins, density=True)
  return hist

def TV_distance(expert_states, student_states):
  expert_hist = get_x_position_histogram(expert_states)
  student_hist = get_x_position_histogram(student_states)
  return 0.5 * np.sum(np.abs(expert_hist - student_hist))


def evaluate(gail):
  """Evaluate the policy learned by GAIL according to three metrics:
    1. Environment reward. We want this number to be large (~100)
    2. How well it fools the discriminator. In particular, we compute the
      discriminator's prediction that the policy is the expert. The policy
      tries to increase this number, while the discriminator tries to decrease
      it. We expect it to be around 40% - 60%
    3. Total variation distance between the student and the expert, along the
      X axis. We want this number to be small (~0)."""
  rewards_vec = []
  p_expert_vec = []
  for _ in range(10):
    states, actions, rewards = generate_episode(gail.env, gail.cmaes.model)
    rewards_vec.append(np.sum(rewards))
    log_p_expert = [gail._reward_fn(s, a) for (s, a) in zip(states, actions)]
    p_expert = np.exp(log_p_expert)
    p_expert_vec.append(np.mean(p_expert))
  tv_dist = TV_distance(gail.expert_states, gail.student_states)
  return np.mean(p_expert_vec), np.mean(rewards_vec), tv_dist

In [0]:
discriminator_update_period = 10

env = gym.make('CartPole-v0')
gail = GAIL(env)
for t in range(100):
  # WRITE CODE HERE

  p_expert, avg_r, tv_dist = evaluate(gail)
  print('(%d) Policy: p(expert) = %.2f%% ; reward = %.1f' % (t, 100.0 * p_expert, avg_r))
  print('(%d) Total variation distance = %.2f' % (t, tv_dist))

# You're Done!